In [1]:
!pip install tensorflow==1.14.0
# preprocessing imports
import pandas as pd
import numpy as np
from time import time
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import text_to_word_sequence

     |████████████████████████████████| 109.2MB 69kB/s 
     |████████████████████████████████| 3.2MB 46.0MB/s 
     |████████████████████████████████| 491kB 50.6MB/s 
  Found existing installation: tensorboard 2.2.0
    Uninstalling tensorboard-2.2.0:
      Successfully uninstalled tensorboard-2.2.0
  Found existing installation: tensorflow-estimator 2.2.0rc0
    Uninstalling tensorflow-estimator-2.2.0rc0:
      Successfully uninstalled tensorflow-estimator-2.2.0rc0
  Found existing installation: tensorflow 2.2.0rc2
    Uninstalling tensorflow-2.2.0rc2:
      Successfully uninstalled tensorflow-2.2.0rc2


Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [2]:
from google.colab import drive 
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [55]:
rawData=pd.read_csv('/content/gdrive/My Drive/Final Year Project/DeepConn/Deep_Learning_Recommender_System/unembedded_grouped_cleaned_data.csv')
rawData.head()

,Unnamed: 0,user_id,food_id,rating,userReviews,foodReviews
0,0,9974,998,5,"['delicious', 'and', 'easy', 'to', 'make']","['delicious', 'and', 'easy', 'to', 'make']"
1,1,10340,998,5,"['feeling', 'healthy', 'great', 'replication',...","['delicious', 'and', 'easy', 'to', 'make']"
2,2,12047,998,5,"['yum', 'will', 'make', 'again']","['delicious', 'and', 'easy', 'to', 'make']"
3,3,13451,998,5,"['turned', 'out', 'great', 'i', 'added', 'pean...","['delicious', 'and', 'easy', 'to', 'make']"
4,8,18226,998,5,"['fantastic', 'tasting', 'completely', 'simple...","['delicious', 'and', 'easy', 'to', 'make']"


In [0]:
# Train/test split for our model is unique, we need to hold out a
# set of users and movies so that our network never learns those 
testSize = 0.005

# get testSize percentage of users
uniqueUsers = rawData.loc[:, "user_id"].unique()
usersSize = len(uniqueUsers)
test_idx = np.random.choice(usersSize,
                              size=int(usersSize * testSize),
                              replace=False)

# get test users
test_users = uniqueUsers[test_idx]

# everyone else is a training user
trainUsers = np.delete(uniqueUsers, test_idx)

test = rawData[rawData["user_id"].isin(test_users)]
train = rawData[rawData["user_id"].isin(trainUsers)]

uniqueTestFood = test["food_id"].unique()

# drop the movies that also appear in our test set. In order to be
# a true train/test split, we are forced to discard some data entirely
train = train.where(np.logical_not(train["food_id"].isin(uniqueTestFood))).dropna()

In [0]:

from sklearn.feature_extraction.text import TfidfVectorizer
embeddingSize=241
tfidf=TfidfVectorizer(max_features=241)
tfidf.fit(list(train["userReviews"]))
tfidf_features=tfidf.transform(list(train["userReviews"])).astype("float64")

trainUserReviews=tfidf_features.toarray()
trainFoodReviews=tfidf.transform(list(train["foodReviews"])).toarray()


testUserReviews=tfidf.transform(list(test["userReviews"])).toarray()
testFoodReviews=tfidf.transform(list(test["foodReviews"])).toarray()

In [58]:
print(type(trainUserReviews),trainUserReviews.shape)

<class 'numpy.ndarray'> (15669, 241)


In [0]:
#!pip install tensorflow==1.14.0
import tensorflow as tf

from keras.models import Model
from keras.callbacks import EarlyStopping, TensorBoard
from keras.layers import Conv1D, MaxPooling1D, Flatten
from keras.layers import Input, Dense , Embedding
from keras.layers.merge import Add, Dot, Concatenate

In [0]:
class DeepCoNN():
    def __init__(self,
                 embeddingSize,
                 hiddenSize,
                 filters=2,
                 kernelSize=10,
                 strides=6):
        self.embeddingSize = embeddingSize
        self.hiddenSize = hiddenSize
        self.filters = filters
        self.kernelSize = kernelSize
        self.inputU, self.towerU = self.createDeepCoNnTower()
        self.inputM, self.towerM = self.createDeepCoNnTower()
        self.joined = Concatenate()([self.towerU, self.towerM])
        self.outNeuron = Dense(1)(self.joined)

    def createDeepCoNnTower(self):
        inputLayer = Input(shape=(self.embeddingSize,))
        embeddingLayer=Embedding(self.embeddingSize,100)(inputLayer)
        tower = Conv1D(filters=self.filters,
                       kernel_size=self.kernelSize,
                       activation="tanh")(embeddingLayer)
        tower = MaxPooling1D()(tower)
        tower = Flatten()(tower)
        tower = Dense(self.hiddenSize, activation="relu")(tower)
        return inputLayer, tower

    def createDeepCoNnDp(self):
        dotproduct = Dot(axes=1)([self.towerU, self.towerM])
        output = Add()([self.outNeuron, dotproduct])
        self.model = Model(inputs=[self.inputU, self.inputM], outputs=[output])
        self.model.compile(optimizer='Adam', loss='mse')
        
    def train(self, trainData,trainUserReviews,trainFoodReviews, batch_size, epochs=1):
        tensorboard = TensorBoard(log_dir="tf_logs/{}".format(pd.Timestamp(int(time()), unit="s")))
        self.createDeepCoNnDp()
        print(self.model.summary())
        
        # userReviews = np.array(trainData.loc[:, "userReviews"])
        # foodReviews = np.array(trainData.loc[:, "foodReviews"])
        userReviews=trainUserReviews
        foodReviews=trainFoodReviews

        self.train_inputs = [userReviews, foodReviews]
        self.train_outputs = trainData.loc[:, "rating"]
        
        self.history = self.model.fit(self.train_inputs,
                                      self.train_outputs,
                                      callbacks=[tensorboard],
                                      validation_split=0.05,
                                      batch_size=batch_size,
                                      epochs=epochs)
        
        

In [64]:
hiddenSize = 64

deepconn = DeepCoNN(trainUserReviews.shape[1], hiddenSize)

batch_size = 32
deepconn.train(train,trainUserReviews,trainFoodReviews, batch_size, epochs=10)

deepconn.model.save("cnn.h5")
# print(train_embedded.loc[0]

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           (None, 241)          0                                            
__________________________________________________________________________________________________
input_14 (InputLayer)           (None, 241)          0                                            
__________________________________________________________________________________________________
embedding_12 (Embedding)        (None, 241, 100)     24100       input_13[0][0]                   
__________________________________________________________________________________________________
embedding_13 (Embedding)        (None, 241, 100)     24100       input_14[0][0]                   
____________________________________________________________________________________________

In [65]:

testInputs = [testUserReviews, testFoodReviews]
print(testInputs)
# dat = pd.DataFrame(testInputs)
# dat.to_csv("/content/gdrive/My Drive/DeepConn/Deep_Learning_Recommender_System//test_data.csv")

trueRating = np.array(list(test.loc[:, "rating"])).reshape((-1, 1))

predictions = deepconn.model.predict(testInputs)

error = np.square(predictions - trueRating)

print("MSE:", np.average(error))

[array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.45865294],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]]), array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])]
MSE: 0.5959017717145865
